# Setup



In [ ]:
# This code works best in google colaboratory. Use google colab for this 
# In this case we will be using Tensor Flow version 2. We will be using the idea of transfer learning on one of the
# pre trained models. We will be doing this as an end to end exercise. 

# We may not need to update the tensorflow 
# Works best with tf==2.6.0 but 2.8.2 is also a good version 
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
# Lets install some basic libraries 
!pip install tf_slim
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 31.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# We will be cloning github tensorflow model repository as we want to do some transfer learning with those models
# on our current dataset. 
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3491, done.
remote: Counting objects: 100% (3491/3491), done.
remote: Compressing objects: 100% (2912/2912), done.
remote: Total 3491 (delta 909), reused 1422 (delta 526), pack-reused 0
Receiving objects: 100% (3491/3491), 46.95 MiB | 30.94 MiB/s, done.
Resolving deltas: 100% (909/909), done.


In [ ]:
# Convert proto files into python 
# Protos conversion
%%bash
cd /content/models/research
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Lets install setup 
%%bash 
cd /content/models/research/object_detection/packages/tf2
pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research/object_detection/packages/tf2
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1236 sha256=1ebf5b54709123c62d68a72b258eaa040c7375c568281a12d25a348bdf0645fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-yszruweh/wheels/d3/af/d9/d9a25faaa2ce4aef17ef6a07f9bb33d386281f8f3580bce973
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=bc12342201f5c24ddc676f10e42999b4ee386b70a83c8281bf8db015a640139b
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=521da2b2e0363ab73c8ae15a77b8085b40f7c1aa45efda9712d2b303d4fbed3e
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for av

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# Download Kaggle datasets 
# Follow this : kaggle.com/general/74235
#kaggle datasets download -d sshikamaru/car-object-detection
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp "/content/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/world-population-dataset                     World Population Dataset                             17KB  2022-08-31 11:20:04          12411        358  1.0              
whenamancodes/hr-employee-attrition                            Employee Analysis | Attrition Report                 50KB  2022-09-12 10:46:33           1037         30  1.0              
pantanjali/unemployment-dataset                                Unemployment dataset                                 17KB  2022-09-08 08:26:10           4201        105  1.0              
whenamancodes/student-performan

In [ ]:
!kaggle datasets download -d sshikamaru/car-object-detection

 96% 108M/112M [00:00<00:00, 135MB/s]  
100% 112M/112M [00:00<00:00, 119MB/s]


In [ ]:
# unzip the dataset
!unzip /content/car-object-detection.zip

Archive:  /content/car-object-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/testing_images/vid_5_25100.jpg  
  inflating: data/testing_images/vid_5_25120.jpg  
  inflating: data/testing_images/vid_5_25140.jpg  
  inflating: data/testing_images/vid_5_25160.jpg  
  inflating: data/testing_images/vid_5_25180.jpg  
  inflating: data/testing_images/vid_5_25200.jpg  
  inflating: data/testing_images/vid_5_25220.jpg  
  inflating: data/testing_images/vid_5_25240.jpg  
  inflating: data/testing_images/vid_5_25260.jpg  
  inflating: data/testing_images/vid_5_26320.jpg  
  inflating: data/testing_images/vid_5_26400.jpg  
  inflating: data/testing_images/vid_5_26420.jpg  
  inflating: data/testing_images/vid_5_26560.jpg  
  inflating: data/testing_images/vid_5_26580.jpg  
  inflating: data/testing_images/vid_5_26600.jpg  
  inflating: data/testing_images/vid_5_26620.jpg  
  inflating: data/testing_images/vid_5_26640.jpg  
  inflating: data/testing_images/vid_5_26660.jp

# Models

## Download pre-trained model

In [ ]:
%cd /content/models/research

/content/models/research


In [ ]:
# Download the model from model zoo fo tensorflow 2
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2022-10-02 06:59:24--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.111.128, 2607:f8b0:4004:c19::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.111.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   190MB/s    in 1.2s    

2022-10-02 06:59:26 (190 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [ ]:
# Extracting the tar file 
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
# Rename the folder as its a long name 
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 retinanet50

In [ ]:
# create a new training folder that will keep the checkpoints, saved models and config file for our new 
# trained model. 

!mkdir training

In [ ]:
# Generally we keep two things in the training folder : one is labelmap.pbtxt and other is config file 
# Now we need to manually create the labelmap.pbtxt file. In this case we just have one class and hence we will create
# one class in our labelmap file. 
'''
item{
  id:1
  name:'car'
}
'''



"\nitem{\n  id:1\n  name:'car'\n}\n"

## Generating TFRecords from CSV 

### Getting the CSV input ready

In [ ]:
# Getting the csv input ready 
df_basic = pd.read_csv(r"/content/data/train_solution_bounding_boxes (1).csv")

In [ ]:
df_basic

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [ ]:
# As there is only one class 
df_basic["class"] = 'car'

In [ ]:
df_basic['width'] = df_basic['xmax']-df_basic['xmin']

In [ ]:
df_basic['height'] = df_basic['ymax'] = df_basic['ymin']

In [ ]:
df_basic

,image,xmin,ymin,xmax,ymax,class,width,height
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,187.035071,car,46.468886,187.035071
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,187.035071,car,105.166425,187.035071
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,176.764801,car,122.775687,176.764801
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,172.363256,car,133.536903,172.363256
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,186.546010,car,115.927641,186.546010
...,...,...,...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,198.321729,car,49.235251,198.321729
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,156.482351,car,206.788055,156.482351
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,168.295823,car,141.797524,168.295823
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,172.233646,car,129.488711,172.233646


In [ ]:
df_basic.rename(columns={'image':'filename'}, inplace = True)
df_basic

,filename,xmin,ymin,xmax,ymax,class,width,height
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,187.035071,car,46.468886,187.035071
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,187.035071,car,105.166425,187.035071
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,176.764801,car,122.775687,176.764801
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,172.363256,car,133.536903,172.363256
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,186.546010,car,115.927641,186.546010
...,...,...,...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,198.321729,car,49.235251,198.321729
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,156.482351,car,206.788055,156.482351
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,168.295823,car,141.797524,168.295823
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,172.233646,car,129.488711,172.233646


In [ ]:
df_basic = df_basic[['filename','width','height','class','xmin','ymin','xmax','ymax']]

## Splitting the total record in test and train data. 

We have 559 rows and we will take 459 as training images and remaining 100 as test images 

In [ ]:
df_basic_test = df_basic[459::]
df_basic_test

,filename,width,height,class,xmin,ymin,xmax,ymax
459,vid_4_6440.jpg,77.285094,192.414736,car,56.740955,192.414736,134.026049,192.414736
460,vid_4_6440.jpg,112.992764,180.188224,car,415.774240,180.188224,528.767004,180.188224
461,vid_4_6440.jpg,75.328509,178.721043,car,534.147612,178.721043,609.476122,178.721043
462,vid_4_6460.jpg,105.655572,191.925676,car,184.408104,191.925676,290.063676,191.925676
463,vid_4_6460.jpg,88.046310,182.144466,car,357.076700,182.144466,445.123010,182.144466
...,...,...,...,...,...,...,...,...
554,vid_4_9860.jpg,49.235251,198.321729,car,0.000000,198.321729,49.235251,198.321729
555,vid_4_9880.jpg,206.788055,156.482351,car,329.876184,156.482351,536.664239,156.482351
556,vid_4_9900.jpg,141.797524,168.295823,car,0.000000,168.295823,141.797524,168.295823
557,vid_4_9960.jpg,129.488711,172.233646,car,487.428988,172.233646,616.917699,172.233646


In [ ]:
df_basic_train = df_basic[0:459]
df_basic_train

,filename,width,height,class,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,46.468886,187.035071,car,281.259045,187.035071,327.727931,187.035071
1,vid_4_10000.jpg,105.166425,187.035071,car,15.163531,187.035071,120.329957,187.035071
2,vid_4_10040.jpg,122.775687,176.764801,car,239.192475,176.764801,361.968162,176.764801
3,vid_4_10020.jpg,133.536903,172.363256,car,496.483358,172.363256,630.020260,172.363256
4,vid_4_10060.jpg,115.927641,186.546010,car,16.630970,186.546010,132.558611,186.546010
...,...,...,...,...,...,...,...,...
454,vid_4_6400.jpg,126.688857,184.100708,car,159.461650,184.100708,286.150507,184.100708
455,vid_4_6400.jpg,99.296672,190.458494,car,346.315485,190.458494,445.612156,190.458494
456,vid_4_6420.jpg,98.807525,184.100708,car,6.358900,184.100708,105.166426,184.100708
457,vid_4_6420.jpg,77.285094,188.502252,car,191.256150,188.502252,268.541245,188.502252


In [ ]:
df_basic_train.to_csv(r"/content/data/train.csv", index = False)
df_basic_test.to_csv(r"/content/data/test.csv", index = False)

In [ ]:
# We need to put the python file called generate_tfrecord.py in our research folder. 
# For the python file : https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py
!python generate_tfrecord.py --csv_input="/content/data/train.csv" --image_dir="/content/data/training_images" --output_path="train.record"

2022-10-02 08:21:26.525712: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-02 08:21:27.207831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:21:27.207935: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:21:27.207954: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p

In [ ]:
# We need to put the python file called generate_tfrecord.py in our research folder. 
# For the python file : https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py
!python generate_tfrecord.py --csv_input="/content/data/test.csv" --image_dir="/content/data/training_images" --output_path="test.record"

2022-10-02 08:21:31.619260: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-02 08:21:32.291558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:21:32.291663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:21:32.291682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p

## Lets train the model now 

In [ ]:
# Before we start training lets edit the config file for the following things 
# change the num_classes to 1 
# change the fine tune checkpoint path to retinanet50/checkpoint/ckpt-0
# change num_steps to 500 reducing number
# change finetune checkpoint type to detection 
# change the labelmap path to labelmap.pbtxt
# train input reader path to train.record
# eval input reader path to test.record
# change the batch size as per your GPU capacity 


In [ ]:
pwd()

'/content/models/research'

In [ ]:
# Installing the latest CUDA version 
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 7s (63.3 MB/s)
(Reading database ... 159447 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 159425 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

In [ ]:
# Executing the model using the pipeline configs 
# put the model_main_tf2.py file outside in object detection folder
# because the path is not detected sometimes 
!python model_main_tf2.py --model_dir=training --pipeline_config_path=training/pipeline.config

2022-10-02 08:57:39.508660: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-02 08:57:41.866526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:57:41.867274: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 08:57:41.867301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p

# Exporting the trained model 

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path training/pipeline.config --trained_checkpoint_dir training --output_directory car_detection/model

2022-10-02 09:10:59.411485: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-02 09:11:00.143780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 09:11:00.143892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-02 09:11:00.143912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p

# Prediction

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
# Importing object detection 
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
import pathlib
def load_custom_model(model_name):
  model_file = model_name
  model_dir = pathlib.Path(model_file)/"saved_model"
  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'training/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/data/testing_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('/content/data/testing_images/vid_5_25100.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25120.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25140.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25160.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25180.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25200.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25220.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25240.jpg'),
 PosixPath('/content/data/testing_images/vid_5_25260.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26320.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26400.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26420.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26560.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26580.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26600.jpg'),
 PosixPath('/content/data/testing_images/vid_5_26620.jpg'),
 PosixPath('/content/data/testing_images

In [ ]:
model_name = '/content/models/research/car_detection/model'
detection_model = load_custom_model(model_name)

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'input_tensor:0' shape=(1, None, None, 3) dtype=uint8>, <tf.Tensor 'unknown:0' shape=() dtype=resource>, <tf.Tensor 'unknown_0:0' shape=() dtype=resource>, <tf.Tensor 'unknown_1:0' shape=() dtype=resource>, <tf.Tensor 'unknown_2:0' shape=() dtype=resource>, <tf.Tensor 'unknown_3:0' shape=() dtype=resource>, <tf.Tensor 'unknown_4:0' shape=() dtype=resource>, <tf.Tensor 'unknown_5:0' shape=() dtype=resource>, <tf.Tensor 'unknown_6:0' shape=() dtype=resource>, <tf.Tensor 'unknown_7:0' shape=() dtype=resource>, <tf.Tensor 'unknown_8:0' shape=() dtype=resource>, <tf.Tensor 'unknown_9:0' shape=() dtype=resource>, <tf.Tensor 'unknown_10:0' shape=() dtype=resource>, <tf.Tensor 'unknown_11:0' shape=() dtype=resource>, <tf.Tensor 'unknown_12:0' shape=() dtype=resource>, <tf.Tensor 'unknown_13:0' shape=() dtype=resource>, <tf.Tensor 'unknown_14:0' shape=() dtype=resource>, <tf.Tensor 'unknown_15:0' shape=() dtype=resource>, <tf.Tensor 'unknown_16:0' shape=() dtype=resource>, <tf.Tenso

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

{'raw_detection_boxes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_anchor_indices': tf.float32,
 'detection_classes': tf.float32,
 'detection_scores': tf.float32}

In [ ]:
detection_model.signatures['serving_default'].output_shapes

{'raw_detection_boxes': TensorShape([1, 51150, 4]),
 'num_detections': TensorShape([1]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'raw_detection_scores': TensorShape([1, 51150, 2]),
 'detection_multiclass_scores': TensorShape([1, 100, 2]),
 'detection_anchor_indices': TensorShape([1, 100]),
 'detection_classes': TensorShape([1, 100]),
 'detection_scores': TensorShape([1, 100])}

# Inferencing the Model

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
#for image_path in TEST_IMAGE_PATHS:
#  show_inference(detection_model, image_path)

In [ ]:
# My model is not predicting anything as I could not train